In [3]:
#!/usr/bin/env python3
# pip install geopandas shapely vt2geojson mercantile tqdm pyproj pandas numpy requests

import time, math, json, requests, mercantile
import numpy as np, pandas as pd, geopandas as gpd
from pathlib import Path
from tqdm import tqdm
from shapely.geometry import LineString, MultiLineString, GeometryCollection
from shapely.ops import unary_union
from vt2geojson.tools import vt_bytes_to_geojson
from pyproj import CRS, Geod

# ───────── CONFIG ─────────
API_KEY  = "BDxJkUVBgwiy5yhJyLyf"
CENTER   = (15.319655248308802, -91.50398347834617)
RADIUS_M = 2000
ZOOM     = 15
LAYER    = "transportation"
TMP       = Path("ejes.geojson")
OUT_UTM   = Path("calzada_buffer.geojson")
OUT_WGS84 = Path("calzada_buffer_wgs84.geojson")

# Diccionario de códigos EPSG UTM por país
EPSG_UTM_BY_COUNTRY = {
    "El Salvador": 32616,
    "Guatemala":   32615,
    "Nicaragua":   32616,
    "Honduras":    32616,
    "Peru":        32718,   # ← Lima: UTM 18 S (hemisferio sur → 327xx)
}

# Selección de país y sistema de coordenadas
pais    = "Guatemala"
crs_utm = CRS.from_epsg(EPSG_UTM_BY_COUNTRY[pais])

NARROW     = 0.72   # recorte global base

DEFAULT_WIDTH = {
    "motorway":32,"trunk":26,"primary":22,"secondary":18,
    "tertiary":14,"residential":10,"service":5,"footway":2
}
LANE_WIDTH = 2.8

# ─────——— DICCIONARIO DE RANGOS → FACTOR —————
RANGE_FACTORS = {
    (0,   3): 0.95,   # 0-3 m  servidumbre / pasaje peatonal muy angosto
    (3,   6): 0.85,   # 3-6 m  callejón / pasaje estrecho
    (6,  10): 0.85,   # 6-10 m calle local estándar
    (10, 16): 0.75,   # 10-16 m avenida secundaria
    (16, 22): 0.65,   # 16-22 m avenida primaria
    (22, 1e9):0.55    # ≥22 m  autopista / muy ancha
}

# ------------------------------------------------

# ───────── DENSIFICAR ─────────
def densify_linestring(ls, max_seg=3.0):
    pts=list(ls.coords); out=[pts[0]]
    for a,b in zip(pts, pts[1:]):
        seg=LineString([a,b]); n=max(1, math.ceil(seg.length/max_seg))
        for k in range(1,n):
            out.append(seg.interpolate(k/n, normalized=True).coords[0])
        out.append(b)
    return LineString(out)

def densify_geom(g, max_seg=3.0):
    if isinstance(g, LineString):
        return densify_linestring(g,max_seg)
    if isinstance(g, MultiLineString):
        return MultiLineString([densify_linestring(ls,max_seg) for ls in g.geoms])
    return g

# ───────── CURVATURA ─────────
def _angles(pts):
    ang=[]
    for i in range(1,len(pts)-1):
        p0,p1,p2=pts[i-1],pts[i],pts[i+1]
        v1=(p1[0]-p0[0],p1[1]-p0[1]); v2=(p2[0]-p1[0],p2[1]-p1[1])
        m=math.hypot(*v1)*math.hypot(*v2)
        if m:
            ang.append(math.degrees(math.acos(max(min((v1[0]*v2[0]+v1[1]*v2[1])/m,1),-1))))
    return ang

def curve_score(g):
    if g.is_empty: return 0
    if isinstance(g, LineString):
        a=_angles(list(g.coords)); return np.percentile(a,90) if a else 0
    if isinstance(g,(MultiLineString,GeometryCollection)):
        all_ang=[ang for p in g.geoms if isinstance(p,LineString) for ang in _angles(list(p.coords))]
        return np.percentile(all_ang,90) if all_ang else 0
    return 0

# ───────── DESCARGA TESSELAS ─────────
t0=time.perf_counter()
geod=Geod(ellps="WGS84")
lons,lats=[],[]
for az in (0,90,180,270):
    lon,lat,_=geod.fwd(CENTER[1],CENTER[0],az,RADIUS_M)
    lons.append(lon); lats.append(lat)
tiles=list(mercantile.tiles(min(lons),min(lats),max(lons),max(lats),[ZOOM]))
features=[]
for t in tqdm(tiles,desc="Descargando"):
    url=f"https://api.maptiler.com/tiles/v3/{t.z}/{t.x}/{t.y}.pbf?key={API_KEY}"
    pbf=requests.get(url,timeout=15,verify=False).content
    features+=vt_bytes_to_geojson(pbf,t.x,t.y,t.z,layer=LAYER)["features"]
TMP.write_text(json.dumps({"type":"FeatureCollection","features":features},ensure_ascii=False))
print(f"↳ Descarga: {time.perf_counter()-t0:0.1f}s")

# ───────── GENERAR BUFFER ─────────
t1=time.perf_counter()
gdf=gpd.read_file(TMP).to_crs(crs_utm); idx=gdf.index

w=pd.to_numeric(gdf.get("width"),errors="coerce")
l=pd.to_numeric(gdf.get("lanes"),errors="coerce")
if not isinstance(w,pd.Series): w=pd.Series(w,index=idx)
if not isinstance(l,pd.Series): l=pd.Series(l,index=idx)

class_s=gdf.get("class").reindex(idx)
gdf["computed_width"]=(w.fillna(l*LANE_WIDTH)
                         .fillna(class_s.map(DEFAULT_WIDTH))
                         .fillna(6))

densified=[]; scores=[]
for geom in tqdm(gdf.geometry,desc="Curvature"):
    d=densify_geom(geom); densified.append(d); scores.append(curve_score(d))
gdf["geometry"]=densified; gdf["curve_score"]=scores

# ───── aplicar diccionario de rangos ─────
ranges_sorted = sorted(RANGE_FACTORS.items(), key=lambda kv: kv[0][0])
edges   = np.array([low for (low,_),_ in ranges_sorted] + [1e9])
factors = np.array([val for _ , val in ranges_sorted] + [ranges_sorted[-1][1]])
gdf["shrink_factor"]=np.interp(gdf["computed_width"], edges, factors)

gdf["curve_factor"]=np.interp(gdf["curve_score"],[0,60],[1.0,0.6]).clip(0.6,1.0)

p95=gdf["computed_width"].quantile(.95)
p99=gdf["computed_width"].quantile(.99)
WIDE_LIMIT=max(p95,22)
MAX_W=max(p99,WIDE_LIMIT+8)
extra=np.interp(gdf["computed_width"],[WIDE_LIMIT,MAX_W],[1.0,0.58])

gdf["eff_shrink"]=gdf["shrink_factor"]*gdf["curve_factor"]*NARROW*extra

buffers=[]
for _,r in tqdm(gdf.iterrows(),total=len(gdf),desc="Buffering"):
    radio=r.computed_width*r.eff_shrink/2
    res=int(np.clip(np.interp(radio*2,[2,40],[8,32]),8,32))
    buffers.append(r.geometry.buffer(radio,cap_style="flat",
                                     join_style="mitre",mitre_limit=3.0,
                                     quad_segs=res//4))

parts=[unary_union(buffers[i:i+400]) for i in range(0,len(buffers),400)]
calzada=unary_union(parts).buffer(0)
calzada=(calzada.buffer(0.2,resolution=4)
                 .buffer(-0.2,resolution=4)
                 .simplify(0.015,preserve_topology=True))
print(f"↳ Buffer+union: {time.perf_counter()-t1:0.1f}s")

gpd.GeoSeries([calzada],crs=crs_utm).to_file(OUT_UTM,driver="GeoJSON")
gpd.GeoSeries([calzada],crs=crs_utm).to_crs(epsg=4326).to_file(OUT_WGS84,driver="GeoJSON")
print("✅ UTM  →",OUT_UTM.resolve())
print("✅ WGS84→",OUT_WGS84.resolve())
print(f"⏱ Total: {time.perf_counter()-t0:0.1f}s")


Descargando:   0%|                                                                             | 0/20 [00:00<?, ?it/s]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:   5%|███▍                                                                 | 1/20 [00:01<00:21,  1.11s/it]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  10%|██████▉                                                              | 2/20 [00:01<0

↳ Descarga: 19.4s


Buffering: 100%|██████████████████████████████████████████████████████████████████| 386/386 [00:00<00:00, 1577.12it/s]


↳ Buffer+union: 14.9s
✅ UTM  → C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer.geojson
✅ WGS84→ C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer_wgs84.geojson
⏱ Total: 35.3s
